In [1]:
#libraries
'''
!pip install transformers==4.31.0 
!pip install sentence-transformers==2.2.2 
!pip accelerate==0.21.0 \
!pip einops==0.6.1 \
!pip langchain==0.0.240 \
!pip xformers==0.0.20 \
'''

'\n!pip install transformers==4.31.0 \n!pip install sentence-transformers==2.2.2 \n!pip accelerate==0.21.0 !pip einops==0.6.1 !pip langchain==0.0.240 !pip xformers==0.0.20 '

In [2]:
#loading test data for response generation
import pandas as pd
test_data_path="C:/Users/USER/Documents/Projects/Instruction Dataset/LLM Training Data/test_data_split.csv"

In [3]:
#processing the test data to add the queries
#defining the queries for the properties-we need to choose a query based on the input
query_temperature="working temperature of the quantum cascade laser"
query_power= "optical power of the quantum cascade laser"
query_frequency="lasing frequency of the quantum cascade laser"
query_hs="heterostructure of the quantum cascade laser"
query_design_type="design type of the quantum cascade laser"

In [4]:
#checking for the query keywords
#checking for the query keywords
def check_text(row, column_name):
    text = row[column_name].lower()
    if "working temperature" in text:
        return query_temperature
    elif "optical power" in text:
        return query_power
    elif "lasing frequency" in text:
        return query_frequency
    elif "query_hs" in text:
        return query_hs
    else:
        return query_design_type

In [5]:
#processing the data
def process_csv(input_filepath, output_filepath, column_name):
    # Load the CSV file
    df = pd.read_csv(input_filepath)

    # Check if the specified column exists
    if column_name in df.columns:
        # Apply the check_text function to the specified column and create a new column with the results
        df['queries'] = df.apply(check_text, axis=1, args=(column_name,))
    else:
        print(f"Column '{column_name}' does not exist in the CSV file.")
        return

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_filepath, index=False)
    print(f"Processed file saved as '{output_filepath}'.")

In [6]:
#processing the test data
input_csv = test_data_path
output_csv = 'processed_test_data.csv'
column_to_check = 'Instruction'
process_csv(input_csv, output_csv, column_to_check)

Processed file saved as 'processed_test_data.csv'.


In [7]:
#loading the processed test data
test_data_df=pd.read_csv('processed_test_data.csv')

In [8]:
#putting the data in some order and renaming the columns
test_data_df=test_data_df[['queries', 'Instruction', 'input_text']]
test_data_df.rename(columns={'Instruction': 'instructions', 'input_text': 'texts'}, inplace=True)

In [9]:
#verifying the ordered data
#test_data_df

In [10]:
#generating a lsit of prompts
prompts=[]
for a,b in zip(test_data_df['instructions'],test_data_df['texts']):
    prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations.

Instruction: {a}

Input Text: {b}

[/INST]
'''
    prompts.append(prompt)

In [11]:
#verifying the list of prompts generated for each row in the dataset
#display the prompt
#prompts

In [12]:
#verify the length of the prompt list
print(len(prompts))

130


In [13]:
#GPT
import openai
import os
from IPython.display import Markdown
api_key="OPENAI_API_KEY"

In [14]:
import openai

# Set your API key securely
openai.api_key = api_key

def chatWithGPT4(user_text, print_output=False):
    try:
        # Creating text completions using the updated `ChatCompletion` class
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",  # Easily switch between "gpt-3.5-turbo-0613", "gpt-4", "text-davinci-004", or others
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": user_text}],
            max_tokens=3000
        )
        text_output = response['choices'][0]['message']['content'] if response['choices'] else "No response generated."
        if print_output:
            print(text_output)
        return text_output
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [15]:
#getting the model responses in a list
responses=[]
for prompt in prompts:
    result=chatWithGPT4(prompt)
    responses.append(result)

In [16]:
#dispalying the responses
responses

['3.3 THz',
 '10 mW',
 '2.07 THz',
 '190 K in pulsed mode and 125 K in continuous wave mode',
 '107 K in pulsed mode, 91 K in continuous wave (cw) mode',
 'none',
 'In0.6Ga0.4As/InAs',
 '3.0 THz',
 '185 K',
 '325 K',
 'none',
 '195 K',
 '24 mW',
 'none',
 '150 K in pulsed mode and 110 K in continuous wave mode',
 'none',
 '113 K in pulsed mode, 89 K in continuous wave (cw) mode',
 '70K',
 'None',
 'resonant-phonon terahertz',
 '182 K',
 'none',
 'GaAs/Al0.15Ga0.85As',
 'none',
 'direct LO-phonon-based depopulation',
 '3.5 THz',
 '115 K, 98 K',
 '13.5 THz',
 '12.4 THz',
 '9.5 THz',
 'GaAs/Al0.15Ga0.85As',
 'scattering-assisted injection and resonant-phonon depopulation',
 '1.88 and 2.42 THz',
 'None',
 '18mW',
 '6 THz',
 '10 mW',
 'GaAs/AlGaAs',
 '5.2 THz',
 '15 mW',
 '8.2 THz',
 '10 milliwatts',
 'Bound-to-continuum',
 '2.12 THz',
 '260 K',
 'bound-to-continuum active region and high-resistivity surface-plasmon waveguide design',
 'resonant-phonon terahertz',
 '119 K',
 '225 K',
 'GaAs

In [17]:
#writing the model responses to a csv file
import csv

# Name of the csv file to save the list
file_name = "GPT4-Turbo-responses.csv"

# Writing the list to a csv file with one column
with open(file_name, "w", newline="") as file:
    writer = csv.writer(file)
    for item in responses:
        writer.writerow([item])

print("List saved to", file_name)

List saved to GPT4-Turbo-responses.csv
